# Try out Embedding
- Faiss
- Chromadb
- Langchain
- https://huggingface.co/BAAI/bge-small-en-v1.5


In [22]:
import pandas as pd
import chromadb

In [19]:
governments = pd.read_json("./government_metadata.json", orient="index")

In [28]:
governments

,summary,full_page_content
Anarchy,Anarchy is a society without rulers and gods.\...,Anarchy is a society without rulers and gods.\...
Aristocracy,Aristocracy (from Ancient Greek ἀριστοκρατίᾱ ...,Aristocracy (from Ancient Greek ἀριστοκρατίᾱ ...
Authoritarianism,Authoritarianism is a political system charact...,Authoritarianism is a political system charact...
Bureaucracy,Bureaucracy ( bure-OK-rə-see) is a body of non...,Bureaucracy ( bure-OK-rə-see) is a body of non...
Capitalism,Capitalism is an economic system based on the ...,Capitalism is an economic system based on the ...
Confederation,A confederation (also known as a confederacy o...,A confederation (also known as a confederacy o...
Colonialism,Colonialism is a practice by which a one group...,Colonialism is a practice by which a one group...
Communism,"Communism (from Latin communis, 'common, unive...","Communism (from Latin communis, 'common, unive..."
Corporatocracy,"Corporatocracy (, from corporate and Greek: -κ...","Corporatocracy (, from corporate and Greek: -κ..."
Democracy,"Democracy (from Ancient Greek: δημοκρατία, rom...","Democracy (from Ancient Greek: δημοκρατία, rom..."


In [23]:
chroma_client = chromadb.PersistentClient(path="/path/to/data")

In [24]:
collection = chroma_client.create_collection(name="governments")

In [34]:
ids_summaries = (governments.index + "_summary").tolist()
summaries = governments["summary"].tolist()
ids_ft = (governments.index + "_fulltext").tolist()
fulltexts = governments["full_page_content"].tolist()

In [47]:
## Adding data to a collection
collection.add(documents=summaries, ids=ids_summaries)
collection.add(documents=fulltexts, ids=ids_ft)

Add of existing embedding ID: Anarchy_summary
Add of existing embedding ID: Aristocracy_summary
Add of existing embedding ID: Authoritarianism_summary
Add of existing embedding ID: Bureaucracy_summary
Add of existing embedding ID: Capitalism_summary
Add of existing embedding ID: Confederation_summary
Add of existing embedding ID: Colonialism_summary
Add of existing embedding ID: Communism_summary
Add of existing embedding ID: Corporatocracy_summary
Add of existing embedding ID: Democracy_summary
Add of existing embedding ID: Electocracy_summary
Add of existing embedding ID: Ergatocracy_summary
Add of existing embedding ID: Fascism_summary
Add of existing embedding ID: Federalism_summary
Add of existing embedding ID: Feudalism_summary
Add of existing embedding ID: Geniocracy_summary
Add of existing embedding ID: Imperialism_summary
Add of existing embedding ID: Kakistocracy_summary
Add of existing embedding ID: Kleptocracy_summary
Add of existing embedding ID: Meritocracy_summary
Add of

In [48]:
results = collection.query(query_texts=["What is capitalism?"], n_results=1)

In [51]:
collection.query(query_texts=["Which form of government cares most about resources?"], n_results=2)

{'ids': [['Ergatocracy_fulltext', 'Technocracy_fulltext']],
 'distances': [[1.109222412109375, 1.119309902191162]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['This article lists forms of government and political systems, according to a series of different ways of categorizing them. The systems listed are not mutually exclusive, and often have overlapping definitions.According to Yale professor Juan José Linz there are three main types of political systems today: democracies, \ntotalitarian regimes and, sitting between these two, authoritarian regimes with hybrid regimes. Another modern classification system includes monarchies as a standalone entity or as a hybrid system of the main three. Scholars generally refer to a dictatorship as either a form of authoritarianism or  totalitarianism.The ancient Greek philosopher Plato discusses in the Republic five types of regimes: aristocracy, timocracy, oligarchy, democracy, and tyranny. \n The question raised by Plato 

In [ ]:
## Updating data from a collection
"""
collection.update(
    ids=["id1", "id2", "id3", ...],
    embeddings=[[1.1, 2.3, 3.2], [4.5, 6.9, 4.4], [1.1, 2.3, 3.2], ...],
    metadatas=[{"chapter": "3", "verse": "16"}, {"chapter": "3", "verse": "5"}, {"chapter": "29", "verse": "11"}, ...],
    documents=["doc1", "doc2", "doc3", ...],
)
"""

In [ ]:
## Langchain + Chroma


In [29]:
## Sentence Transformers

In [30]:
## Huggingface